In [32]:
import os
import sys

pwd = os.path.sep.join(os.getcwd().split(sep=os.path.sep)[:-1])
sys.path.append(pwd)

import dataset.dataset as dataset
import model.model as model
import torch
import torch.nn as nn
import torch.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.Size([1, 18])